<a href="https://colab.research.google.com/github/mohamedabdelhameed711/ITI-NLP-tasks/blob/main/task_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install spacy datasets


In [ ]:
from datasets import load_dataset
import spacy
from spacy.tokens import Doc, DocBin
from datasets import load_dataset


dataset = load_dataset("conll2003", trust_remote_code=True)

# Get NER tag names
label_list = dataset["train"].features["ner_tags"].feature.names

def create_spacy_docs(dataset_split, nlp):
    doc_bin = DocBin()
    for example in dataset_split:
        tokens = example["tokens"]
        ner_tags = example["ner_tags"]

        # Create Doc from tokens
        doc = Doc(nlp.vocab, words=tokens)
        ents = []
        i = 0
        while i < len(ner_tags):
            tag = label_list[ner_tags[i]]
            if tag.startswith("B-"):
                start = i
                ent_label = tag[2:]
                i += 1
                while i < len(ner_tags) and label_list[ner_tags[i]] == f"I-{ent_label}":
                    i += 1
                end = i
                span = doc.char_span(doc[start].idx, doc[end - 1].idx + len(doc[end - 1]), label=ent_label, alignment_mode="expand")
                if span:
                    ents.append(span)
            else:
                i += 1
        doc.ents = ents
        doc_bin.add(doc)
    return doc_bin


nlp = spacy.blank("en")

create_spacy_docs(dataset["train"], nlp).to_disk("train.spacy")
create_spacy_docs(dataset["validation"], nlp).to_disk("dev.spacy")


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy debug config config.cfg

In [ ]:
! python -m spacy init fill-config config.cfg config.cfg


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
import spacy

nlp = spacy.load("output/model-best")
doc = nlp("Barack Obama visited Berlin in 2008.")
print([(ent.text, ent.label_) for ent in doc.ents])
